In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/SIR_Origin/*.csv'
dic_SIRs = Load_files.load_files(path, -17, -15)

In [3]:
key = 'SIR'
var_list = ['up1', 'up2', 'down1', 'down2']
cities = ['강원', '경기', '경남', '경북', '광주', '대구', 
          '대전', '부산', '서울', '세종', '울산', '인천',
          '전남', '전북', '제주', '충남', '충북']
models = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
          'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 
          'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

# SIR

In [4]:
var_num = 0
dic_total_model_final = {}
dic_model_rank = {}
dic_city_model_rank = {}

for name in models:
    dic_model_rank[name] = 0

dic_model_no_rank = dic_model_rank.copy()

for var in var_list:
    
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)

    dic_hyperparameter = {}
    for city in cities:
        path = f'hyperparameter/Stricgency/{key}/{var}/{city}/*.pkl'
        files = glob.glob(path)
        hyperparameter = {}
        for file in files:
            if var_num < 2:
                hyperparameter[file[44:-4]] = load_hyperparameter(file)
            else:
                hyperparameter[file[48:-4]] = load_hyperparameter(file)
                
        dic_hyperparameter[city] = hyperparameter
        
    city_predicted = {}

    for city in cities:

        dic_model = load_model_multiple(dic_hyperparameter, 'Stricgency', key, var, city)
        data = dic_files[city]
        df = Prepare_df.processing(data, 'Date', 'alpha')

        x = df.iloc[:, 0:]
        y = df.iloc[:,:1]

        ms = MinMaxScaler()
        ss = StandardScaler()

        ss.fit(x)
        ms.fit(y)

        predicted_values = {}
        num_model = 0
        for name, state in dic_model.items():
            batch_size = 64
            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 238, 60, 1, batch_size, 'mto')
                label_y, predicted = predict_mto(state, df, x_ss, y_ms)
                predicted_values[name] = [label_y, predicted]
                
                data_predicted = data[['Date']].copy()

                data_predicted['alpha'] = data['alpha']

                data_predicted['predicted_alpha'] = 0
                data_predicted.iloc[-1*len(predicted):, -1] = predicted

                data_predicted['Infected'] = data['Infected']

                data_predicted['predicted_Infected'] = 0
                data_predicted['predicted_Infected'] = (data['Infected'].shift().fillna(0) 
                                                              * (1 + data['alpha']*data['Susceptible']/dic_SIRs[city].iloc[0, 2] 
                                                                - data['beta'] - data['gamma']))
            
                for i in range(len(data_predicted)-13, len(data_predicted)):

                    data_predicted.iloc[i, -1] = (data_predicted.iloc[i-1, -1]
                                                  * (1 + data_predicted.iloc[i-1, 2]*data.iloc[i-1, 3]/dic_SIRs[city].iloc[0, 2]
                                                    - data.iloc[i-1, 8] - data.iloc[i-1, 9]))

                data_predicted = data_predicted[62:].reset_index(drop = True)
                actual_Infected = data_predicted['Infected']
                predicted_Infected = data_predicted['predicted_Infected']

                predicted_values[name] = [actual_Infected, predicted_Infected]
            
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 225, 60, 14, batch_size, 'mtm')
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(state, df, x_ss, y_ms, 14, 0.5, device)
                predicted_values[name] = [label_y, predicted, first_label_y, first_predicted, all_predicted]

                data_predicted = data[['Date']].copy()

                data_predicted['alpha'] = data['alpha']

                data_predicted['predicted_alpha'] = 0
                data_predicted.iloc[-1*len(all_predicted):, -1] = all_predicted

                data_predicted['Infected'] = data['Infected']

                data_predicted['predicted_Infected'] = 0
                data_predicted['predicted_Infected'] = (data['Infected'].shift().fillna(0) 
                                                        * (1 + data['alpha']*data['Susceptible']/dic_SIRs[city].iloc[0, 2] 
                                                            - data['beta'] - data['gamma']))

                for i in range(len(data_predicted)-13, len(data_predicted)):

                    data_predicted.iloc[i, -1] = (data_predicted.iloc[i-1, -1]
                                                  * (1 + data_predicted.iloc[i-1, 2]*data.iloc[i-1, 3]/dic_SIRs[city].iloc[0, 2]
                                                    - data.iloc[i-1, 8] - data.iloc[i-1, 9]))

                data_predicted = data_predicted[62:].reset_index(drop = True)
                actual_Infected = data_predicted['Infected']
                predicted_Infected = data_predicted['predicted_Infected']

                predicted_values[name] = [actual_Infected, predicted_Infected]
         
            num_model += 1

        city_predicted[city] = predicted_values

    dic_best_model = {}
    dic_total_model = {}

    for city in cities:

        actual_data = city_predicted[city]['RNN'][0]
        num_model = 0
        dic_model = {}

        for name, file in city_predicted[city].items():
            predicted_data = file[1]

            rmse_val = rmse(actual_data[-14:], predicted_data[-14:])
            mape_val = mape(actual_data[-14:], predicted_data[-14:])
            dic_model[name] = [rmse_val, mape_val, actual_data, predicted_data]

            num_model += 1

        dic_total_model[city] = dic_model
    dic_total_model_final[var] = dic_total_model
    var_num += 1
    
    dic_var_model_rank = {}
    for city in cities:
        dic_model_rank_sub = dic_model_no_rank.copy()
        rmse_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][0])).keys())
        for m in rmse_rank:
            dic_model_rank[m] += rmse_rank.index(m)
            dic_model_rank_sub[m] += rmse_rank.index(m)
        mape_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][1])).keys())
        for m in mape_rank:
            dic_model_rank[m] += mape_rank.index(m)
            dic_model_rank_sub[m] += mape_rank.index(m)
        dic_var_model_rank[city] = list(dict(list(sorted(dic_model_rank_sub.items(), key = lambda x:x[1]))).keys())
    dic_city_model_rank[var] = dic_var_model_rank

In [5]:
dic_city_model_rank['up1']['서울']

['seq2seq_LSTM',
 'RNN',
 'BiRNN',
 'seq2seq_BiRNN',
 'LSTM',
 'seq2seq_GRU',
 'GRU',
 'BiGRU',
 'seq2seq_RNN',
 'BiLSTM',
 'seq2seq_BiGRU',
 'seq2seq_BiLSTM']

In [6]:
dict(sorted(dic_model_rank.items(), key = lambda x:x[1]))

{'BiRNN': 572,
 'BiLSTM': 658,
 'BiGRU': 671,
 'LSTM': 678,
 'seq2seq_RNN': 691,
 'RNN': 697,
 'seq2seq_LSTM': 700,
 'GRU': 721,
 'seq2seq_BiRNN': 722,
 'seq2seq_BiLSTM': 918,
 'seq2seq_GRU': 956,
 'seq2seq_BiGRU': 992}

In [7]:
model_rank_list = list(dict(sorted(dic_model_rank.items(), key = lambda x:(x[1]))).keys())
model_rank_list

['BiRNN',
 'BiLSTM',
 'BiGRU',
 'LSTM',
 'seq2seq_RNN',
 'RNN',
 'seq2seq_LSTM',
 'GRU',
 'seq2seq_BiRNN',
 'seq2seq_BiLSTM',
 'seq2seq_GRU',
 'seq2seq_BiGRU']

In [8]:
top_model = model_rank_list[:6]
top_model

['BiRNN', 'BiLSTM', 'BiGRU', 'LSTM', 'seq2seq_RNN', 'RNN']

In [9]:
dic_var_diff = {}

for var in dic_total_model_final:
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)
    dic_city_diff = {}
    for city in cities:
        data = dic_files[city]
        index_min = np.inf
        dic_model_diff = {}
        print(var, city)
        for name in top_model:
            rmse_val =  rmse(dic_total_model_final[var][city][name][2][-14:], dic_total_model_final[var][city][name][3][-14:])
            mape_val =  mape(dic_total_model_final[var][city][name][2][-14:], dic_total_model_final[var][city][name][3][-14:])
            diff_val = float(sum(dic_total_model_final[var][city][name][2][-14:] - dic_total_model_final[var][city][name][3][-14:]))
            index_val = dic_city_model_rank[var][city].index(name)
            dic_model_diff[name] = [rmse_val, mape_val, diff_val]
            #plotting(dic_total_model[city][t][1], dic_total_model[city][t][2], 132)
            if index_min > index_val:
                index_min = index_val
                best_rmse = rmse_val
                best_mape = mape_val
                best_diff = diff_val
                best_model = name
                best_predicted = dic_total_model_final[var][city][name][3]
                
            df = pd.concat([dic_total_model_final[var][city][name][2], dic_total_model_final[var][city][name][3]], axis = 1)
            df.index = list(data['Date'][-1*len(actual_data):])
            df.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/{name}.csv", encoding = 'cp949')
        dic_model_diff['best_model'] = [best_model, best_rmse, best_mape, best_diff]
        dic_city_diff[city] = dic_model_diff
        
        df_best = pd.concat([dic_total_model_final[var][city][name][2], best_predicted], axis = 1)
        df_best.index = list(data['Date'][-1*len(actual_data):])
        df_best.to_csv(f"Data/Predicted/Stricgency/{key}/{var}/{city}/best_{best_model}.csv", encoding = 'cp949')
        print(best_model, best_rmse, best_mape, best_diff)
    dic_var_diff[var] = dic_city_diff
    
with open(f'Data/Predicted/Stricgency/{key}/valid_score.pkl', 'wb') as f:
    pickle.dump(dic_var_diff, f)

up1 강원
BiLSTM 29.90607822050725 7.913389705385747 -107.86460861136203
up1 경기
BiLSTM 91.23486180157748 2.311527099216429 1035.8666066080855
up1 경남
BiLSTM 12.317279434996701 2.7879857156269954 -94.10239249760372
up1 경북
BiLSTM 49.11819752004462 10.505640163198612 337.4739251659069
up1 광주
BiRNN 43.4307972443355 11.549043859933096 301.8931490586908
up1 대구
BiRNN 34.30143950081644 8.804370878020569 371.95149666596336
up1 대전
BiRNN 13.629677451484495 8.915546660346456 -116.11515772026698
up1 부산
RNN 153.74675618300114 26.51017394285938 -1545.8374047494913
up1 서울
RNN 423.77845167415177 9.583444528406652 -5136.886216297078
up1 세종
seq2seq_RNN 2.2167234772876543 11.272415595130283 1.4291746407302952
up1 울산
BiRNN 19.44060267125619 7.953669146446367 210.10110852133107
up1 인천
BiRNN 19.361626393662895 2.3719281889087553 -22.37372191451709
up1 전남
BiGRU 6.828038661384576 9.043966146569158 78.0602152939291
up1 전북
seq2seq_RNN 37.2573254356284 21.924818781337336 -486.2626367734962
up1 제주
BiGRU 7.986164925547

# SIR_alpha

In [4]:
var_num = 0
dic_total_model_final = {}
dic_model_rank = {}
dic_city_model_rank = {}

for name in models:
    dic_model_rank[name] = 0

dic_model_no_rank = dic_model_rank.copy()

for var in var_list:
    
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)

    dic_hyperparameter = {}
    for city in cities:
        path = f'hyperparameter/Stricgency/{key}/{var}/{city}/*.pkl'
        files = glob.glob(path)
        hyperparameter = {}
        for file in files:
            if var_num < 2:
                hyperparameter[file[44:-4]] = load_hyperparameter(file)
            else:
                hyperparameter[file[48:-4]] = load_hyperparameter(file)
                
        dic_hyperparameter[city] = hyperparameter
        
    city_predicted = {}

    for city in cities:

        dic_model = load_model_multiple(dic_hyperparameter, 'Stricgency', key, var, city)
        data = dic_files[city]
        df = Prepare_df.processing(data, 'Date', 'alpha')

        x = df.iloc[:, 0:]
        y = df.iloc[:,:1]

        ms = MinMaxScaler()
        ss = StandardScaler()

        ss.fit(x)
        ms.fit(y)

        predicted_values = {}
        num_model = 0
        for name, state in dic_model.items():
            batch_size = 64
            if num_model < 6:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 238, 60, 1, batch_size, 'mto')
                label_y, predicted = predict_mto(state, df, x_ss, y_ms)
                predicted_values[name] = [label_y, predicted]  
            else:
                x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 225, 60, 14, batch_size, 'mtm')
                label_y, predicted, first_label_y, first_predicted, all_predicted = predict_mtm(state, df, x_ss, y_ms, 14, 0.5, device)
                predicted_values[name] = [label_y, predicted, first_label_y, first_predicted, all_predicted]
         
            num_model += 1

        city_predicted[city] = predicted_values

    dic_best_model = {}
    dic_total_model = {}

    for city in cities:

        actual_data = city_predicted[city]['RNN'][0]
        num_model = 0
        dic_model = {}

        for name, file in city_predicted[city].items():
            predicted_data = file[1]

            rmse_val = rmse(actual_data[-14:], predicted_data[-14:])
            mape_val = mape(actual_data[-14:], predicted_data[-14:])
            dic_model[name] = [rmse_val, mape_val, actual_data, predicted_data]

            num_model += 1

        dic_total_model[city] = dic_model
    dic_total_model_final[var] = dic_total_model
    var_num += 1
    
    dic_var_model_rank = {}
    for city in cities:
        dic_model_rank_sub = dic_model_no_rank.copy()
        rmse_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][0])).keys())
        for m in rmse_rank:
            dic_model_rank[m] += rmse_rank.index(m)
            dic_model_rank_sub[m] += rmse_rank.index(m)
        mape_rank = list(dict(sorted(dic_total_model[city].items(), key = lambda x:x[1][1])).keys())
        for m in mape_rank:
            dic_model_rank[m] += mape_rank.index(m)
            dic_model_rank_sub[m] += mape_rank.index(m)
        dic_var_model_rank[city] = list(dict(list(sorted(dic_model_rank_sub.items(), key = lambda x:x[1]))).keys())
    dic_city_model_rank[var] = dic_var_model_rank

In [9]:
dic_var_diff = {}

for var in dic_total_model_final:
    path = f'Data/Stricgency/{key}/{var}/*.csv'
    dic_files = Load_files.load_files(path, -17, -15)
    dic_city_diff = {}
    for city in cities:
        data = dic_files[city]
        index_min = np.inf
        dic_model_diff = {}
        print(var, city)
        for name in top_model:
            rmse_val =  rmse(dic_total_model_final[var][city][name][2][-14:], dic_total_model_final[var][city][name][3][-14:])
            mape_val =  mape(dic_total_model_final[var][city][name][2][-14:], dic_total_model_final[var][city][name][3][-14:])
            diff_val = float(sum(dic_total_model_final[var][city][name][2][-14:] - dic_total_model_final[var][city][name][3][-14:]))
            index_val = dic_city_model_rank[var][city].index(name)
            dic_model_diff[name] = [rmse_val, mape_val, diff_val]
            #plotting(dic_total_model[city][t][1], dic_total_model[city][t][2], 132)
            if index_min > index_val:
                index_min = index_val
                best_rmse = rmse_val
                best_mape = mape_val
                best_diff = diff_val
                best_model = name
                best_predicted = dic_total_model_final[var][city][name][3]
                
            df = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][2], 
                                              dic_total_model_final[var][city][name][3]], 
                                             axis = 1),
                              columns = ['Actual_data', 'Predicted_data'])
            df.index = list(data['Date'][-1*len(actual_data):])
            df.to_csv(f"Data/Predicted/Stricgency/{key}_alpha/{var}/{city}/{name}.csv", encoding = 'cp949')
        dic_model_diff['best_model'] = [best_model, best_rmse, best_mape, best_diff]
        dic_city_diff[city] = dic_model_diff
        df_best = pd.DataFrame(np.concatenate([dic_total_model_final[var][city][name][2], 
                                               best_predicted], 
                                              axis = 1),
                               columns = ['Actual_data', 'Predicted_data'])
        df_best.index = list(data['Date'][-1*len(actual_data):])
        df_best.to_csv(f"Data/Predicted/Stricgency/{key}_alpha/{var}/{city}/best_{best_model}.csv", encoding = 'cp949')
        print(best_model, best_rmse, best_mape, best_diff)
    dic_var_diff[var] = dic_city_diff
    
with open(f'Data/Predicted/Stricgency/{key}_alpha/valid_score.pkl', 'wb') as f:
    pickle.dump(dic_var_diff, f)

up1 강원
BiRNN 0.03246663182770913 52.73818373680115 0.15363910794258118
up1 경기
BiLSTM 0.013162055015487091 18.625447154045105 0.05342057719826698
up1 경남
BiRNN 0.041111276842761585 48.660650849342346 0.07577131688594818
up1 경북
BiLSTM 0.12029051486939277 161.25047206878662 -0.650269627571106
up1 광주
GRU 0.07057265987502015 107.97477960586548 -0.007692040875554085
up1 대구
GRU 0.026407161848912182 43.049004673957825 0.02315596491098404
up1 대전
GRU 0.034672142502894715 118.80013942718506 -0.05325066298246384
up1 부산
GRU 0.05145564811464689 87.24855184555054 -0.25085902214050293
up1 서울
RNN 0.021288771774992946 44.648146629333496 -0.2134944051504135
up1 세종
LSTM 0.18565203046024817 inf 0.0732579231262207
up1 울산
BiRNN 0.061146259276253896 112.05990314483643 0.06433764100074768
up1 인천
BiRNN 0.025064293556192947 37.56113946437836 -0.014466416090726852
up1 전남
LSTM 0.07705647081320588 inf 0.5741164684295654
up1 전북
BiRNN 0.08606961239061447 161.44152879714966 -0.3788299560546875
up1 제주
BiLSTM 0.022529981